In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [ ]:
# Load the pre-trained ResNet model
model = models.resnet18(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])  # Remove the last fully connected layer

# Set the model in evaluation mode (important when using pre-trained models)
model.eval()



In [ ]:
# Preprocessing function to transform the image into a tensor
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

# Function to create image embeddings
def create_image_embedding(image_path):
    try:
        input_tensor = preprocess_image(image_path)
        with torch.no_grad():
            embeddings = model(input_tensor)
        return embeddings.squeeze().numpy()
    except Exception as e:
        print("Error:", e)
        return None

In [ ]:
# Example usage
image_path = "./images/cat1.jpg"
cat1 = create_image_embedding(image_path)

# 'embedding' now contains a dense vector representation of the image
print("Image Embedding Shape:", cat1.shape)
print("Image Embedding:", cat1)




In [ ]:
image_path = "./images/cat2.jpg"
cat2 = create_image_embedding(image_path)

# 'embedding' now contains a dense vector representation of the image
print("Image Embedding Shape:", cat2.shape)
print("Image Embedding:", cat2)



In [ ]:
dog1 = create_image_embedding("./images/dog1.jpg")
dog2 = create_image_embedding("./images/dog2.jpg")
dog3 = create_image_embedding("./images/dog3.jpg")
person1 = create_image_embedding("./images/person1.jpg")
person2 = create_image_embedding("./images/person2.jpg")

In [ ]:
person1.shape

In [ ]:
#Imports a PyMilvus package:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [ ]:
#Connect to the Milvus
connections.connect("default", host="localhost", port="19530")

In [ ]:
#Creates a collection:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="words", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=512)
]
schema = CollectionSchema(fields, "Simple Demo for image similar search")
image = Collection("image", schema)

In [ ]:
# Builds indexes on the entities:

index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

image.create_index("embeddings", index)

In [ ]:
#Insert data in collection
data = [
    [1,2,3,4,5,6,7],  # field pk
    ['cat1','cat2','dog1','dog2','dog3','person1','person2'],  # field words
    [cat1, cat2, dog1, dog2, dog3, person1, person2],  # field embeddings
]

In [ ]:
image.insert(data)
image.flush()
image.load()

In [ ]:
search_params = {"metric_type": "L2"}

In [ ]:
results = image.search(
	data=[dog1], 
	anns_field="embeddings", 
	param=search_params,
	limit=4, 
	expr=None,
	# set the names of the fields you want to retrieve from the search result.
	output_fields=['words'],
	consistency_level="Strong"
)

In [ ]:
results[0].ids

In [ ]:
results[0].distances

In [ ]:
hit = results[0][3]
hit.entity.get('words')

In [ ]:
results[0][0].entity.get('words')

In [ ]:
from PIL import Image

In [ ]:
for i in range(0,len(results[0])):
    name = results[0][i].entity.get('words')
    print(name)
    display(Image.open('./images/'+name+'.jpg'))

In [ ]:
results = image.search(
	data=[cat1], 
	anns_field="embeddings", 
	param=search_params,
	limit=4, 
	expr=None,
	# set the names of the fields you want to retrieve from the search result.
	output_fields=['words'],
	consistency_level="Strong"
)

In [ ]:
for i in range(0,len(results[0])):
    name = results[0][i].entity.get('words')
    print(name)
    display(Image.open('./images/'+name+'.jpg'))

In [ ]:
Binod1 = create_image_embedding("./images/Binod1.jpg")
Binod2 = create_image_embedding("./images/Binod2.jpg")
Binod3 = create_image_embedding("./images/Binod3.jpg")
Binod4 = create_image_embedding("./images/Binod4.jpg")



In [ ]:
Binod1.shape

In [ ]:
#Insert data in collection
data = [
    [8,9,10,11],  # field pk
    ['Binod1','Binod2','Binod3','Binod4'],  # field words
    [Binod1,Binod2,Binod3,Binod4],  # field embeddings
]

In [ ]:
image.insert(data)
image.flush()
image.load()

In [ ]:
image.release()

In [ ]:
image.load()

In [ ]:
expr = "pk in [6,7]"
image.delete(expr)

In [ ]:
results = image.search(
	data=[Binod2], 
	anns_field="embeddings", 
	param=search_params,
	limit=4, 
	expr=None,
	# set the names of the fields you want to retrieve from the search result.
	output_fields=['words'],
	consistency_level="Strong"
)

In [ ]:
for i in range(0,len(results[0])):
    name = results[0][i].entity.get('words')
    print(name)
    display(Image.open('./images/'+name+'.jpg'))

In [ ]:
harsh1 = create_image_embedding("./images/harsh1.jpg")
harsh2 = create_image_embedding("./images/harsh2.jpg")
harsh3 = create_image_embedding("./images/harsh3.jpg")
harsh4 = create_image_embedding("./images/harsh4.jpg")




In [ ]:
#Insert data in collection
data = [
    [12,13,14,14],  # field pk
    ['harsh1','harsh2','harsh3','harsh4'],  # field words
    [harsh1,harsh2,harsh3,harsh4],  # field embeddings
]
image.insert(data)
image.flush()
image.load()

In [ ]:
results = image.search(
	data=[harsh2], 
	anns_field="embeddings", 
	param=search_params,
	limit=4, 
	expr=None,
	# set the names of the fields you want to retrieve from the search result.
	output_fields=['words'],
	consistency_level="Strong"
)

In [ ]:
for i in range(0,len(results[0])):
    name = results[0][i].entity.get('words')
    print(name)
    display(Image.open('./images/'+name+'.jpg'))